## Flask-Admin:
extension that adds admin interface generation capabilities to Flask applications. It automatically creates a user interface for managing your data models, while allowing extensive customization when needed.

In [ ]:
!pip install flask-admin

### Project Structure

flask-admin-app/<br>
├── app/<br>
│   ├── __init__.py<br>
│   ├── admin/<br>
│   │   ├── __init__.py<br>
│   │   ├── views.py<br>
│   │   ├── forms.py<br>
│   │   └── models.py<br>
│   ├── templates/<br>
│   │   ├── admin/<br>
│   │   │   ├── master.html<br>
│   │   │   ├── index.html<br>
│   │   │   ├── custom/<br>
│   │   │   │   ├── dashboard.html<br>
│   │   │   │   └── analytics.html<br>
│   │   │   └── models/<br>
│   │   │       ├── user.html<br>
│   │   │       └── product.html<br>
│   │   └── security/<br>
│   ├── static/<br>
│   │   ├── admin/<br>
│   │   │   ├── css/<br>
│   │   │   └── js/<br>
│   │   └── img/<br>
│   └── models/<br>
├── config.py<br>
└── requirements.txt<br>

### Types of Admin Interfaces


#### 1. Model-based Admin
- Automatic CRUD operations
- Database model management
- File management

#### 2. Custom Admin Views
- Dashboard analytics
- User management
- Content moderation

#### 3. Multi-tenant Admin
- Organization-specific views
- Role-based access
- Resource isolation

### Basic Admin Setup

In [ ]:
# app/__init__.py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_admin import Admin
from flask_admin.contrib.sqla import ModelView
from flask_security import Security, SQLAlchemyUserDatastore,current_user

db = SQLAlchemy()
admin = Admin(template_mode='bootstrap4')

def create_app():
    app = Flask(__name__)
    app.config.from_object('config.Config')
    
    db.init_app(app)
    admin.init_app(app)
    
    class UserModelView(ModelView):
        # Specify Access permissions
        def is_accessible(self):
            return current_user.is_admin

    
    class ProductModelView(ModelView):
        # Specify Access permissions
        def is_accessible(self):
            return current_user.is_admin
        
    # Add model views
    from .models import User, Product
    admin.add_view(UserModelView(User, db.session))
    admin.add_view(ProductModelView(Product, db.session))
    
    return app

### Custom Admin Views

In [ ]:
# app/admin/views.py
from flask_admin import BaseView, expose
from flask_security import current_user, roles_required

class DashboardView(BaseView):
    @expose('/')
    @roles_required('admin')
    def index(self):
        stats = self.get_dashboard_stats()
        return self.render('admin/custom/dashboard.html', stats=stats)
    
    def get_dashboard_stats(self):
        return {
            'total_users': User.query.count(),
            'active_users': User.query.filter_by(active=True).count(),
            'revenue': self.calculate_revenue()
        }

# Register custom view
admin.add_view(DashboardView(name='Dashboard', endpoint='admin_dashboard'))

### Model Views with Custom Actions

In [ ]:
# app/admin/models.py
class UserAdmin(ModelView):
    # Display foreign keys or no 
    column_display_fk = True
    
    # Which columns will be displayed in the list view of the admin interface for users.
    column_list = ('email', 'name', 'active', 'roles')
    
    # Which columns are searchable in the admin interface. In this case, you can search for users by their email or name.
    column_searchable_list = ('email', 'name')
    
    # Which columns can be used to filter results in the list view. Admins can filter users based on their active status and their roles.
    column_filters = ('active', 'roles')
    
    #  Excludes specific fields from the form used to create or edit users.
    form_excluded_columns = ('password_hash',)
    
    # control whether the admin panel allows users to be created, edited, or deleted.
    can_create = True
    can_edit = True
    can_delete = True
    
    # Custom Templates for List and Edit Views
    list_template = 'admin/custom_list.html'
    edit_template = 'admin/custom_edit.html'
        
    # Specify Access permissions
    def is_accessible(self):
        return current_user.has_role('admin')
    
    # Process to do if its not accessible
    def inaccessible_callback(self, name, **kwargs):
        return redirect(url_for('/admin'))
     
    @action('activate', 'Activate Users', 'Are you sure?')
    def action_activate(self, ids):
        try:
            query = User.query.filter(User.id.in_(ids))
            count = 0
            for user in query.all():
                user.active = True
                count += 1
            db.session.commit()
            flash(f'Successfully activated {count} users.')
        except Exception as ex:
            flash(f'Failed to activate users: {str(ex)}', 'error')

- **Custom Action**: This section defines a custom action in the admin panel that allows the admin to activate multiple users at once. The @action decorator defines the action:
    - **activate**: The name of the action.
    - **Activate Users**: The label shown in the admin panel.
    - **Are you sure?**: A confirmation message that appears before the action is executed.

- **Function Logic**:
- The action receives a list of user IDs (ids) selected in the admin panel.
- A query is made to find all users with IDs in the provided list.
- For each user found, the active status is set to True.
- After modifying the users, changes are committed to the database, and a success message is shown using Flask's flash function.
- If there’s an error (e.g., database issue), an error message is shown.


### Custom Forms and Validation

In [ ]:
# app/admin/forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, SelectField, BooleanField
from wtforms.validators import DataRequired, Email

class UserForm(FlaskForm):
    name = StringField('Name', validators=[DataRequired()])
    email = StringField('Email', validators=[DataRequired(), Email()])
    role = SelectField('Role', choices=[
        ('user', 'User'),
        ('admin', 'Admin'),
        ('moderator', 'Moderator')
    ])
    active = BooleanField('Active')

# Using custom form in ModelView
class CustomUserAdmin(ModelView):
    form = UserForm
    
    def on_model_change(self, form, model, is_created):
        if is_created:
            model.set_password(form.password.data)
        super().on_model_change(form, model, is_created)

- **form = UserForm**: By setting the form attribute to UserForm, the CustomUserAdmin class tells Flask-Admin to use the custom UserForm when creating or editing a user. This means when you view the admin interface for users, you'll see the form fields (name, email, role, and active) that were defined in the UserForm.

- **on_model_change**: This method is part of Flask-Admin's ModelView. It gets called every time a model (user) is created or updated via the admin interface.
    - **Parameters**:
        - **form**: The form object that is submitted by the user in the admin interface.
        - **model**: The database model being edited or created.
        - **is_created**: A boolean indicating whether the model is being created (True) or updated (False).
    - **Custom Logic**:
        If the model is being created (is_created == True), this method will set the user's password. It assumes that the password field is part of the form (even though it's not shown in the current form definition, it could be added). The password would be retrieved from the form (form.password.data), and then the model's set_password method would hash and store it.